<h1><b>Statistique en Bioinformatique : </b> TME 4  et 5 </h1>
<br>
L’objectif de ce TME est: 
<br>
<ul>
<li> implémenter l'algorithme de Viterbi et l'estimation des paramètres (en utilisant le Viterbi training)
pour l'exemple du occasionally dishonest casino.   </li> 
</ul>
<br>
<div class="alert alert-warning" role="alert" style="margin: 10px">
<p>**Soumission**</p>
<ul>
<li>Renomer le fichier TME4_5.ipynb pour TME4_5_NomEtudiant1_NomEtudiant2.ipynb </li>
<li>Soumettre à https://www.dropbox.com/request/ZylCDDpggbrN5toTiJKV </li>
</ul>

</div>

Nom etudiant 1 :
<br>
Nom etudiant 2 :
<br>

<h3>Introduction</h3>
Un casino parfois malhonnête (occasionally dishonest casino) utilise 2 types de pieces : fair et unfair. <br>
La matrice de transition entre les états cachés est:<br>
${\cal S}=\{F,U\}$ (fair, unfair):
$$
p = \left(
\begin{array}{cc}
0.99 & 0.01\\
0.05 & 0.95
\end{array}
\right)\ ,
$$

les probabilités d'éemission des symboles 
${\cal O} = \{H,T\}$ (head, tail):
\begin{eqnarray}
e_F(H) =  0.5 &\ \ \ \ &
e_F(T) = 0.5 \nonumber\\
e_U(H) = 0.9 &\ \ \ \ &
e_U(T) = 0.1 \nonumber
\end{eqnarray}

<br> Et la condition initiale $\pi^{(0)} = (1,0)$ (le jeux commence toujours avec le pieces juste (fair).

<b>Exercice 1</b>:
<u>Simulation</u>: Ecrire une fonction qui simule $T$ jets de pieces. 
La fonction renverra un tableau à deux colonnes correspondant 
aux valeurs simulées pour les états cachés $X_t$ 
(type de dés utilisée, “F” ou “U”) et aux symboles observées $Y_t$ 
(résultat du jet de dés, “H” ou “T”). On simulera une séquence
de longueur 2000 qu'on gardera pour les applications ultérieures.


In [1]:
import numpy as np
import matplotlib.pyplot as plt


S = {0: "F", 1: "U"}
Pij = np.array([[0.99, 0.01], [0.05, 0.95]])

O = {0: "H", 1: "T"}
Eij = np.array([[0.5, 0.5], [0.9, 0.1]])  # ça aurait dû être Eio

# Condition initiale
pi0 = np.array([0.999, 0.001])

T = 2000

In [2]:
import random

def simule(pi0,Eij,Pij,T):
    s = np.where(np.cumsum(pi0)>random.random())[0][0]
    table = np.zeros((T,2), dtype=np.int64)
    for i in range(T-1):
        throw = np.where(np.cumsum(Eij[s])>random.random())[0][0]
        table[i]=[s,throw]
        s=np.where(np.cumsum(Pij[s])>random.random())[0][0]
    return table

seq=simule(pi0,Eij,Pij,T)
print(seq)

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 1]
 [0 0]]


<b>Exercice 2</b>: <u>Algorithme de Viterbi </u>: Ecrire une fonction qui permet
de déterminer la séquence $(i^\star_t)_{t=0:T}$ d'états cachés
plus probable, ansi que sa probabilité. Pour tester votre fonction utiliser le résultat de la 
simulation (2éme colonne) de la question 1. Comparer $(i^\star_t)_{t=0:T}$ avec
les vrais états cachés (1ère colonne de la simulation). 


In [3]:
# Algorithme de Viterbi
def viterbi(obs, Pi, A, B):
    """
    Parameters
    ----------
    obs : array (T,)
        Sequence d'observations.
    Pi: array, (K,)
        Distribution de probabilite initiale
    A : array (K, K)
        Matrice de transition
    B : array (K, M)
        Matrice d'emission matrix

    """
    ## initialisation
    psi = np.zeros((len(A), len(obs)))  # A = N
    psi[:, 0] = -1
    delta = np.zeros(
        (len(A), len(obs))
    ) 

    for i in range(len(psi[:, 0])):
        delta[i, 0] = np.log(Pi[i]) + np.log(B[i, obs[0]])

    ## fin initialisation

    for t in range(1, delta.shape[1]):
        for e in range(delta.shape[0]):

            values = np.zeros(delta.shape[0])

            for i in range(delta.shape[0]):
                values[i] = delta[i, t - 1] + np.log(A[i, e])

            val_max = np.amax(values)
            index_max = np.argmax(values)

            delta[e, t] = val_max + np.log(B[e, obs[t]])

            psi[e, t] = index_max

    C = np.zeros(len(obs))
    C[-1] = np.argmax(delta[:, -1])

    for i in range(len(obs) - 2, -1, -1):
        C[i] = psi[C[i + 1].astype("int"), i + 1]

    return C



print(list(viterbi(seq[:,1], pi0, Pij, Eij)))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

<b>Exercice 3</b>: <u>Estimation des paramètres</u>
<br>
3.1) Ecrire une fonction qui utilise tous les résultats de la simulation
(états et symboles) pour compter les nombres d'occurrence $N_{ij}$ est $M_{iO}$ définis
en cours. Estimer $p_{ij}$ est $e_i(O)$, voir slides  37-39 dans la presentation. Attention, pour eviter les probabilites à zero nous alons utiliser les pseudo-count.

In [4]:
# Estimation de Parametres par contage
def param(seq,size_states,size_symbols):
    hidden_states=seq[:,0]
    obs = seq[:,1]

    A= np.ones((size_states,size_states))
    B = np.zeros((size_states, size_symbols))
    for i in range(1,len(hidden_states)-1):
        p=(hidden_states[i-1],hidden_states[i])
        A[p[0],p[1]] +=1
    A = A/np.maximum(A.sum(1).reshape(size_states, 1), 1)
    
    for i in range(len(obs)):
        B[hidden_states[i], obs[i]] += 1

    for b in range(len(B)):
        B[b, :] = B[b, :] / B[b, :].sum()
    return A,B
A,B = param(seq,2,2)
print(A)
print(B)

[[0.99323181 0.00676819]
 [0.05240175 0.94759825]]
[[0.51212634 0.48787366]
 [0.90748899 0.09251101]]


3.2) <u> Viterbi training </u>: Ecrire une fonction qui utilise 
seulement la séquence $(O_t)_{t=0:T}$ (2emme colone de la simulation) pour estimer les 
paramètres $p_{ij}$ est $e_i(O)$. On s'arretera quand les diferences entre les logVraissamblance est inferieur à 1e-04. Comparer les résultats de 3.1 et de 3.2 (3.2 avec plusieurs restarts,
et avec initialisation des paramètres alèatoire).


In [9]:
import matplotlib.pyplot as plt
obs=seq[:,0]
# Initialisation aleatoire de Pij, Eij, pi0
Estimate_Pij=np.log(np.random.rand(2,2))
Estimate_Eij=np.log(np.random.rand(2,2))
Estimate_pi0=np.log(np.random.rand(2))

# Calcule log Vraissamblance
alpha = np.zeros((2,len(obs)))
alpha[0,0]= Estimate_pi0[0]+Estimate_Eij[0,obs[0]]
alpha[1,0]= Estimate_pi0[1]+Estimate_Eij[1,obs[0]]
for i in range(1,len(obs)):
    alpha[0,i]=(alpha[0,i-1]+Estimate_Pij[0,0]+alpha[1,i-1]+Estimate_Pij[1,0])+Estimate_Eij[0,obs[i]]
    alpha[1,i]= (alpha[0,i-1]+Estimate_Pij[0,1]+alpha[1,i-1]+Estimate_Pij[1,1])+Estimate_Eij[1,obs[i]]
log_vraissamblance= np.sum(alpha[:,len(obs)-1])
# Viterbi Training

/home/saighi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in double_scalars
  del sys.path[0]
/home/saighi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in double_scalars
  


3.3) <u>Viterbi training deuxiemme version </u> Ecrivez une version de 3.3 qui:
- part plusieurs fois (100x) d'une initialisation aléatoire des 
paramètres de l'HMM,
- utilise Viterbi training pour estimer les paramètres,
- calcule la log-vraisemblance pour les paramètres estimés,
- sauvegarde seulement l'estimation avec la valeur maximale de la
log-vraisemblance.

Qu'est-ce que vous observez?



In [ ]:
# Viterbi Training  deuxiemme version